In [1]:
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPool2D
from keras.layers import Activation,Dropout,Flatten,Dense
from keras.optimizers import RMSprop,SGD
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16
import h5py
import os



In [ ]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))

nb_train_samples = 18047
nb_val_samples = 4377
image_size = (224,224)
batch_size = 128
class_num = 10

In [14]:
def build_simple_convnet(input_shape=(224,224,3),class_num=class_num):
    
    model = Sequential()
    model.add(Conv2D(32,(3,3),input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Conv2D(128,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(class_num))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [ ]:
def generate_batches_data(dir_path, 
                    gen=image.ImageDataGenerator(), 
                    target_size=(224,224), 
                    batch_size=32, 
                    class_mode='categorical', 
                    shuffle=False):
    
    return gen.flow_from_directory(dir_path, 
                                   target_size=target_size, 
                                   batch_size=batch_size,
                                   class_mode=class_mode, 
                                   shuffle=shuffle)

In [15]:

train_datagen = image.ImageDataGenerator(rescale=1./255,
                                         shear_range=0.2,
                                         rotation_range=10,
                                         height_shift_range=0.1,
                                         width_shift_range=0.1,
                                         channel_shift_range=5,
                                         horizontal_flip=True,
                                         fill_mode='nearest')

validation_datagen = image.ImageDataGenerator(rescale=1./255)

# test_datagen = image.ImageDataGenerator(rescale=1./255)


train_generator = generate_batches_data(
                                    dir_path=train_path,
                                    gen=train_datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = generate_batches_data(
                                    dir_path=validation_path,
                                    gen=validation_datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)


Found 18047 images belonging to 10 classes.
Found 4377 images belonging to 10 classes.


In [16]:
model = build_simple_convnet()
model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=10,
                    callbacks=[TensorBoard()],
                    validation_data=validation_generator,
                    validation_steps=nb_val_samples // batch_size)

model.save_weights('simple_cnn_weights.h5')

Epoch 1/3
1127/1127 [==============================] - 3890s 3s/step - loss: 1.9015 - acc: 0.2967 - val_loss: 2.1572 - val_acc: 0.2999
Epoch 2/3
1127/1127 [==============================] - 3818s 3s/step - loss: 1.2567 - acc: 0.5433 - val_loss: 2.7569 - val_acc: 0.3210
Epoch 3/3
1127/1127 [==============================] - 3941s 3s/step - loss: 0.9579 - acc: 0.6594 - val_loss: 3.0042 - val_acc: 0.3346


In [23]:
#load VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)
# vgg16_model = VGG16(weights=None, include_top=True)

# x = base_model.output
# x = Dense(256,activation='relu')(x)
# x = Dropout(0.5)(x)
# top_model = base_model.output
# top_model = Dense(256,activation='relu')(top_model)
# top_model= Dropout(0.5)(top_model)
# top_model = Dense(256,activation='relu')(top_model)
# top_model= Dropout(0.5)(top_model)
# pred = Dense(10,activation='softmax')(top_model)
# vgg16_model = Model(inputs=base_model.input, outputs=pred)
# vgg16_model.summary()

# x = Sequential()

# for i,layer in enumerate(base_model.layers):
#     if i>18:break
#     x.add(layer)
#     print(i,layer.name)


# top_model = Sequential()
# top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
# top_model.add(Dense(256, activation='relu'))
# top_model.add(Dropout(0.5))
# top_model.add(Dense(10, activation='softmax'))
x = base_model.output
x = Flatten(input_shape=base_model.output_shape[1:])(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(class_num, activation='softmax')(x)

vgg16_model = Model(inputs=base_model.input, outputs=predictions)
vgg16_model.summary()

0 input_23
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


ValueError: The shape of the input to "Flatten" is not fully defined (got (None, None, 512). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.

In [ ]:
for layer in vgg16_model.layers[:25]:
    layer.trainable = False
    
vgg16_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
vgg16_model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=10,
                    callbacks=[TensorBoard()],
                    validation_data=validation_generator,
                    validation_steps=nb_val_samples // batch_size)
vgg16_model.save_weights('vgg16_fc_weights.h5')

In [ ]:
for layer in vgg16_model.layers[15:25]:
    layer.trainable = True

vgg16_model.compile(optimizer=SGD(lr=1e-4,momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])
vgg16_model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=10,
                    callbacks=[TensorBoard()],
                    validation_data=validation_generator,
                    validation_steps=nb_val_samples // batch_size)

vgg16_model.save_weights('vgg16_fine_tuning_weights.h5')